In [1]:
# Cell [1] - Imports and Setup
# iTrust/iTrust/01_requirements_to_artifacts_xml.ipynb

import os
import sys
import logging
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
from typing import Dict, List, Optional, Tuple

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('requirements_to_artifacts')

def handle_exception(func):
    """Decorator to handle exceptions in functions"""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            logger.error(f"Error in {func.__name__}: {str(e)}", exc_info=True)
            raise
    return wrapper

@handle_exception
def prettify_xml(elem: ET.Element) -> str:
    """Convert XML element to pretty-printed string"""
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="    ")

@handle_exception 
def validate_xml(xml_string: bytes) -> bool:
    """Validate XML string"""
    try:
        ET.fromstring(xml_string)
        return True
    except ET.ParseError as e:
        logger.error(f"XML validation failed: {str(e)}")
        return False

In [2]:
# Cell [2] - XML Structure Functions
# iTrust/iTrust/01_requirements_to_artifacts_xml.ipynb

@handle_exception
def create_xml_structure(collection_id: str, collection_name: str, collection_desc: str) -> ET.Element:
    """Create base XML structure for artifacts collection"""
    logger.debug(f"Creating XML structure for collection: {collection_id}")
    
    root = ET.Element("artifacts_collection")
    
    collection_info = ET.SubElement(root, "collection_info")
    ET.SubElement(collection_info, "id").text = collection_id
    ET.SubElement(collection_info, "name").text = collection_name
    ET.SubElement(collection_info, "version").text = "1.1"
    ET.SubElement(collection_info, "description").text = collection_desc
    ET.SubElement(collection_info, "content_location").text = "external"
    
    ET.SubElement(root, "artifacts")
    
    return root

@handle_exception
def add_artifact(artifacts_elem: ET.Element, artifact_id: str, content: str, parent_id: str = "") -> None:
    """Add artifact to XML structure"""
    logger.debug(f"Adding artifact: {artifact_id} with content: {content}")
    
    artifact = ET.SubElement(artifacts_elem, "artifact")
    ET.SubElement(artifact, "id").text = artifact_id
    ET.SubElement(artifact, "content").text = content
    ET.SubElement(artifact, "parent_id").text = parent_id

In [3]:
    # Cell [3] - Requirements Processing Functions
    # iTrust/iTrust/01_requirements_to_artifacts_xml.ipynb

    @handle_exception
    def read_requirement_file(filepath: str) -> Optional[str]:
        """Read and return the content of a requirement file
        
        Args:
            filepath: Path to the requirement file
            
        Returns:
            Content of the file or None if error
        """
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                logger.debug(f"Read file {filepath} - Content length: {len(content)} chars")
                return content
        except Exception as e:
            logger.error(f"Error reading requirement file {filepath}: {str(e)}")
            return None

    @handle_exception
    def extract_requirement_id_uc(filename: str) -> Optional[str]:
        """Extract requirement ID from UC filename
        
        Args:
            filename: Name of the file to process
            
        Returns:
            Requirement ID or None if invalid format
        """
        match = re.match(r'(UC\d+(?:E\d+|S\d+)?).txt', filename)
        if match:
            logger.debug(f"Extracted requirement ID: {match.group(1)} from {filename}")
            return match.group(1)
        logger.warning(f"Invalid UC filename format: {filename}")
        return None

    @handle_exception
    def process_requirements(source_dir: str, output_file: str) -> None:
        """Process UC requirements and create XML file with file contents
        
        Args:
            source_dir: Directory containing UC files
            output_file: Path to output XML file
        """
        logger.info(f"Processing UC requirements from directory: {source_dir}")
        
        root = create_xml_structure(
            collection_id="UC",
            collection_name="iTrust Source Artifacts",
            collection_desc="Use cases"
        )
        artifacts_elem = root.find("artifacts")
        
        try:
            # Ensure source directory exists
            if not os.path.exists(source_dir):
                raise FileNotFoundError(f"Source directory not found: {source_dir}")
                
            # Process each file in the directory
            file_count = 0
            for filename in sorted(os.listdir(source_dir)):
                if filename.lower().endswith('.txt'):
                    filepath = os.path.join(source_dir, filename)
                    logger.info(f"Processing UC file: {filepath}")
                    
                    req_id = extract_requirement_id_uc(filename)
                    if not req_id:
                        continue
                    
                    # Read the actual content of the file
                    content = read_requirement_file(filepath)
                    if content:
                        logger.debug(f"Adding artifact {req_id} - Content preview: {content[:100]}...")
                        add_artifact(artifacts_elem, req_id, content)
                        file_count += 1
                    else:
                        logger.warning(f"Skipping file due to empty/unreadable content: {filepath}")
            
            logger.info(f"Processed {file_count} requirement files")
            
            # Generate and save XML
            xml_string = prettify_xml(root)
            if validate_xml(xml_string.encode('utf-8')):
                with open(output_file, 'w', encoding='utf-8') as f:
                    # Write XML without extra declaration
                    f.write(xml_string)
                logger.info(f"Successfully created requirements XML file: {output_file}")
            else:
                logger.error("Failed to validate XML output")
                
        except Exception as e:
            logger.error(f"Error processing requirements: {str(e)}", exc_info=True)
            raise

In [4]:
    # Cell [4] - Java Code Processing Functions
    # iTrust/iTrust/01_requirements_to_artifacts_xml.ipynb

    @handle_exception
    def read_java_file(filepath: str) -> Optional[str]:
        """Read and return the content of a Java file
        
        Args:
            filepath: Path to the Java file
            
        Returns:
            Content of the file or None if error
        """
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                logger.debug(f"Read Java file {filepath} - Content length: {len(content)} chars")
                return content
        except Exception as e:
            logger.error(f"Error reading Java file {filepath}: {str(e)}")
            return None

    @handle_exception
    def extract_class_name(filepath: str) -> Optional[str]:
        """Extract Java class name from filepath and add CC_ prefix
        
        Args:
            filepath: Path to Java file
            
        Returns:
            Class name with CC_ prefix or None if invalid format
        """
        match = re.search(r'([A-Za-z0-9_]+)\.java$', filepath)
        if match:
            class_name = f"CC_{match.group(1)}"
            logger.debug(f"Extracted class name with prefix: {class_name} from {filepath}")
            return class_name
        logger.warning(f"Invalid Java filename format: {filepath}")
        return None

    @handle_exception
    def process_java_code(source_dir: str, output_file: str) -> None:
        """Process Java code files and create XML file with file contents
        
        Args:
            source_dir: Directory containing Java code
            output_file: Path to output XML file
        """
        logger.info(f"Processing Java code from directory: {source_dir}")
        
        root = create_xml_structure(
            collection_id="CC",
            collection_name="iTrust Target Artifacts",
            collection_desc="Class code"
        )
        artifacts_elem = root.find("artifacts")
        
        try:
            # Ensure source directory exists
            if not os.path.exists(source_dir):
                raise FileNotFoundError(f"Source directory not found: {source_dir}")
                
            # Walk through directory tree
            file_count = 0
            for root_dir, _, files in os.walk(source_dir):
                for filename in sorted(files):
                    if filename.endswith('.java'):
                        filepath = os.path.join(root_dir, filename)
                        logger.info(f"Processing Java file: {filepath}")
                        
                        class_name = extract_class_name(filename)
                        if not class_name:
                            continue
                        
                        # Read the actual content of the file
                        content = read_java_file(filepath)
                        if content:
                            logger.debug(f"Adding Java class {class_name} - Content preview: {content[:100]}...")
                            add_artifact(artifacts_elem, class_name, content)
                            file_count += 1
                        else:
                            logger.warning(f"Skipping file due to empty/unreadable content: {filepath}")
            
            logger.info(f"Processed {file_count} Java files")
            
            # Generate and save XML
            xml_string = prettify_xml(root)
            if validate_xml(xml_string.encode('utf-8')):
                with open(output_file, 'w', encoding='utf-8') as f:
                    # Write XML without extra declaration
                    f.write(xml_string)
                logger.info(f"Successfully created Java code XML file: {output_file}")
            else:
                logger.error("Failed to validate XML output")
                
        except Exception as e:
            logger.error(f"Error processing Java code: {str(e)}", exc_info=True)
            raise

In [ ]:
# Cell [5] - Main Execution
# iTrust/iTrust/01_requirements_to_artifacts_xml.ipynb

def verify_file_creation(filepath: str) -> None:
    """Verify that a file was created and log its size"""
    if os.path.exists(filepath):
        size = os.path.getsize(filepath)
        logger.info(f"Successfully created {filepath} (size: {size} bytes)")
    else:
        logger.error(f"Failed to create {filepath}")

# Process requirements
requirements_dir = "UC"
source_artifacts = "iTrust-sourceArtifacts.xml"

process_requirements(
    source_dir=requirements_dir,
    output_file=source_artifacts
)
verify_file_creation(source_artifacts)

# Process Java code
java_code_dir = "itrust_v10_code/iTrust/src"
target_artifacts = "iTrust-targetArtifacts.xml"

process_java_code(
    source_dir=java_code_dir,
    output_file=target_artifacts
)
verify_file_creation(target_artifacts)

logger.info("Completed processing both requirements and Java code")